In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
from tactile_learning.datasets import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [3]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps, view_num=0): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, f'cam_{view_num}_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [4]:

def dump_states(root, tactile_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [5]:
import glob
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_7',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_9',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_15',
# ]
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling'
# data_path = '/data/tactile_learning/deployment_data/data/bowl_picking/demonstrations/image_generalization'
roots = glob.glob(f'{data_path}/demonstration_*')
# roots = [f'{data_path}/demonstration_6']

retrieval_fps = 1
view_num = 0
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps, view_num=view_num)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices, view_num=view_num)


tactile_indices: [0, 99, 197, 295, 392, 490, 588, 686, 785, 884, 982, 1079, 1177, 1275, 1372, 1470, 1568, 1667, 1765, 1863, 1961, 2060, 2157, 2255, 2354, 2453, 2550, 2649, 2745, 2843, 2941, 3040, 3140, 3238, 3333, 3431, 3530, 3628, 3726, 3825, 3922, 4020, 4118, 4216, 4314, 4410, 4509, 4608, 4706, 4804, 4901, 5000, 5098, 5195, 5294], image_indices: [70, 100, 128, 157, 186, 215, 243, 271, 299, 328, 357, 385, 414, 443, 472, 499, 528, 557, 586, 616, 646, 676, 705, 735, 764, 794, 823, 853, 883, 912, 938, 968, 997, 1026, 1056, 1086, 1115, 1145, 1175, 1204, 1233, 1262, 1291, 1319, 1349, 1379, 1408, 1437, 1467, 1495, 1524, 1554, 1584, 1613, 1642]


100%|██████████| 55/55 [01:06<00:00,  1.20s/it]


tactile_indices: [0, 99, 198, 296, 394, 493, 591, 689, 787, 886, 983], image_indices: [70, 99, 129, 158, 187, 216, 245, 274, 304, 331, 360]


100%|██████████| 11/11 [00:13<00:00,  1.19s/it]


tactile_indices: [0, 100, 198, 296, 394, 492, 589, 686, 785, 883, 982, 1080, 1178, 1276, 1375, 1474, 1572, 1670, 1769, 1867, 1963, 2062, 2160, 2258, 2356, 2454, 2553, 2650, 2749, 2846, 2944, 3042, 3140, 3238, 3335, 3433, 3531, 3628, 3726], image_indices: [71, 100, 130, 159, 187, 217, 246, 276, 305, 334, 364, 393, 423, 452, 481, 511, 540, 570, 598, 628, 658, 688, 718, 748, 777, 807, 837, 866, 896, 925, 953, 983, 1011, 1040, 1069, 1099, 1128, 1158, 1186]


...

100%|██████████| 64/64 [01:19<00:00,  1.24s/it]


tactile_indices: [0, 100, 198, 297, 395, 494, 593, 690, 790, 888, 987, 1085, 1183, 1282, 1379, 1477, 1575, 1674, 1773, 1872, 1969, 2068, 2165, 2263, 2362, 2460, 2556, 2654, 2751, 2849, 2947, 3046, 3144, 3240, 3342, 3436, 3536, 3633, 3730, 3828, 3926, 4023, 4121, 4219, 4318, 4415, 4513, 4611, 4709, 4806, 4905, 5003, 5101, 5198, 5304, 5396, 5494, 5602, 5690, 5787, 5887, 5984, 6084, 6181, 6279, 6378, 6474, 6573, 6671, 6769, 6867, 6966, 7063, 7161, 7259, 7357, 7455, 7553, 7652, 7748, 7846, 7944, 8041, 8139, 8238, 8336, 8432, 8531, 8630, 8728, 8825, 8924, 9022, 9120, 9218, 9317, 9414, 9513, 9611], image_indices: [69, 98, 126, 156, 185, 214, 243, 273, 303, 333, 363, 393, 421, 451, 481, 511, 541, 570, 599, 629, 659, 688, 717, 747, 776, 806, 836, 865, 894, 924, 954, 984, 1013, 1042, 1071, 1101, 1130, 1160, 1190, 1220, 1249, 1279, 1309, 1337, 1367, 1396, 1425, 1455, 1484, 1514, 1543, 1571, 1601, 1631, 1661, 1691, 1721, 1750, 1780, 1810, 1839, 1869, 1899, 1929, 1958, 1988, 2018, 2048, 2077, 2106

100%|██████████| 99/99 [02:04<00:00,  1.26s/it]


tactile_indices: [0, 98, 196, 294, 392, 491, 589, 688, 786, 884, 982, 1081, 1179, 1278, 1375, 1474, 1571, 1668, 1766, 1864, 1961, 2059, 2158, 2257, 2354, 2452, 2554, 2648, 2745, 2842, 2940, 3038, 3136, 3234, 3333, 3432, 3530, 3627, 3725, 3824, 3921, 4020, 4119, 4216, 4315, 4413, 4510, 4606, 4705, 4803, 4902, 5000, 5098, 5196, 5294, 5392, 5490, 5589, 5687, 5784, 5882, 5981], image_indices: [68, 98, 127, 156, 182, 211, 239, 268, 297, 327, 356, 386, 415, 445, 475, 505, 532, 561, 591, 621, 650, 680, 709, 738, 768, 797, 826, 855, 885, 915, 945, 973, 1003, 1032, 1062, 1091, 1120, 1149, 1179, 1208, 1237, 1264, 1294, 1323, 1351, 1381, 1411, 1440, 1469, 1498, 1528, 1558, 1587, 1615, 1645, 1673, 1703, 1732, 1762, 1792, 1822, 1852]


100%|██████████| 62/62 [01:58<00:00,  1.91s/it]


In [6]:
print(roots)

['/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_8', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_3', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_1', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_2', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_9', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_6', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_7', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_5', '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_4']


In [7]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_8


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_3


[libx264 @ 0x5605e9e6c240] using SAR=1/1
[libx264 @ 0x5605e9e6c240] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5605e9e6c240] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x5605e9e6c240] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstr

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_1


...

[libx264 @ 0x55fd1d7de700] using SAR=1/1
[libx264 @ 0x55fd1d7de700] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55fd1d7de700] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x55fd1d7de700] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstr

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_5


[libx264 @ 0x5614dceb41c0] using SAR=1/1
[libx264 @ 0x5614dceb41c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5614dceb41c0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x5614dceb41c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstr

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstration_4


[libx264 @ 0x559b15f35180] using SAR=1/1
[libx264 @ 0x559b15f35180] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x559b15f35180] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x559b15f35180] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/mouse_scrolling/demonstr

In [8]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
